In [3]:
import torch
import sklearn

In [13]:
print("dsasdd")
print("dsasdd")

dsasdd
dsasdd


In [ ]:
print("32")

In [ ]:
print("屬層曦356243234242343423423443232423423423423u48253678345632656253jgfsdfgjgsiufghfudsh6fufhsdfiushfhsuifhshffisfs6")

In [ ]:
print("dsa6565sdd")

In [ ]:
print("dsa6sdd")

In [2]:
print("dsasdd")

dsasdd


In [4]:
import numpy as np
from sklearn.impute import SimpleImputer

# Example numerical data with missing values
data = np.array([[1, 2, np.nan],
                 [3, np.nan, 4],
                 [5, 6, 7]])

# Create an imputer object without specifying a strategy (defaults to 'mean')
imputer = SimpleImputer()

# Fit and transform the data
data_imputed = imputer.fit_transform(data)

print("Original Data:")
print(data)
print("\nImputed Data:")
print(data_imputed)


Original Data:
[[ 1.  2. nan]
 [ 3. nan  4.]
 [ 5.  6.  7.]]

Imputed Data:
[[1.  2.  5.5]
 [3.  4.  4. ]
 [5.  6.  7. ]]


In [6]:
class Parent:
  def __init__(self, txt):
    self.message = txt

  def printmessage(self):
    print(self.message)

class Child(Parent):
  def __init__(self, txt):
    super().__init__(txt)

x = Child("Hello, and welcome!")

x.printmessage()

Hello, and welcome!
